In [15]:
from urllib import request

import requests
from bs4 import BeautifulSoup
import re
import os

baseurl="https://www.bqkan8.com/1_1094/"
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36"}
content=re.compile("read2\(\);</script>(.*?)<script>app2\(\)")
filepath=r"C:/Users/fh282/爬虫练习/books"
def main():
    try:
        linklist=getlinklist(baseurl)
        datalist,charlist=getdata(linklist)
        savedata(datalist,charlist,filepath)
    except Exception as e:
        print(e)
    finally:
        print("执行完毕")


def getlinklist(baseurl):
    linklist=[]
    res=requests.get(baseurl,headers=headers)
    res.encoding="gbk"
    html=res.text
    soap=BeautifulSoup(html,"lxml")
    for item in soap.find_all("dd"):
        link=item.a["href"]+"+"+item.a.text
        linklist.append(link)
    return linklist

def getdata(linklist):
    datalist=[]
    charlist=[]
    for link in linklist[:10]:
        herf,charname=link.split("+")
        url="https://www.bqkan8.com/"+herf
        res=requests.get(url,headers=headers)
        res.encoding="gbk"
        html=res.text
        soap=BeautifulSoup(html,"lxml")
        for item in soap.find_all("div",id="content",class_="showtxt"):
            item=str(item)
            data=re.findall(content,item)[0].replace("<br/>","\n").replace("\xa0","")
            datalist.append(data)
            charlist.append(charname)
    return datalist,charlist

def savedata(datalist,charlist,filepath):
    if "books" not in os.listdir():
        os.makedirs("books")
    for i in range(len(charlist)):
        filename="第"+str(i+1)+"章"
        charpath=filepath+"/"+filename+".txt"
        data=datalist[i]
        f=open(charpath,"w+")
        f.write(data)
        f.close()
   
if __name__=="__main__":
    main()